In [ ]:
%run appV2_dataCleaning.ipynb

In [ ]:
import xgboost as xgb

In [ ]:
import pyforest
from nltk.stem import WordNetLemmatizer 
def lemma(text):
    tokens = nltk.word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(w,'v') for w in tokens]
    return lemmatized

tfidf=TfidfVectorizer(tokenizer= lemma, min_df=3, lowercase = True, ngram_range=(1,2), stop_words='english')
features = tfidf.fit_transform(df.final).toarray()
labels = df.result
features.shape

In [ ]:
#labels.unique()
#lablels = labels.replace({1:0, 2:1, 3:2}, inplace=True)

In [ ]:
indices_test

In [ ]:
sum(labels)/len(labels)

In [ ]:
X_train, X_valid, y_train, y_valid, indices_train, indices_test = train_test_split(features, labels, df.index, test_size=0.20, random_state=0, stratify=labels)


In [ ]:
sum(y_valid)/len(y_valid)

In [ ]:
print(np.unique(np.array(y_valid)))
print(np.unique(np.array(y_train)))
print(np.unique(np.array(labels)))
print(len(np.array(y_valid)))
print(len(np.array(X_train)))

In [ ]:
wt_map = {0: 1, 1: 8, 2: 2} 
y_train_weigth=y_train.map(wt_map)
y_train_weigth

In [ ]:
y_test_weigth=y_valid.map(wt_map)
y_test_weigth

In [ ]:
#train_data = lgb.Dataset(X_train, label=y_train)
#valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
dtrain = xgb.DMatrix(X_train, label=y_train, weight = y_train_weigth)
dtest = xgb.DMatrix(X_valid, label=y_valid, weight = y_test_weigth)

## Baseline Model: 
we can get an f1 score with no effort. Hope we can beat it with parameter tuning. 

In [ ]:
#SEARCH_PARAMS = {'learning_rate': 0.4,
#                'max_depth': 15,
#                'num_leaves': 32,
#                'feature_fraction': 0.8,
#                'subsample': 0.2
#                }
#scale_pos_weight: a weight multiplication on the positive labelThe higher the weight, 
#the greater penalty is imposed on errors on the minor class.
#Therefore, in order to have an unbiased model, errors on the minor class need 
#to be penalised more severely.

SEARCH_PARAMS = {'max_depth': 3, #default 6
                 'subsample': 0.5, #default ,
                 #'n_estimator' : 100, #default 
                 'learning_rate' : 0.1,
                 'min_child_weight' : 1}

FIXED_PARAMS={'objective': 'multi:softprob',
              'eval_metric': None,
              'metric': 'None',
              'num_class': 3 ,
              'random_state': 5,
              'num_boost_round':300,
              'early_stopping_rounds':30}

params = {#'eval_metric':FIXED_PARAMS['eval_metric'],
          'num_class':FIXED_PARAMS['num_class'],
          'objective':FIXED_PARAMS['objective'],  
          'random_state': FIXED_PARAMS['random_state'],
          'disable_default_eval_metric': 1,
          **SEARCH_PARAMS}


In [ ]:
t = np.arange(0, 1, 0.005)
f = np.repeat(0, 200)
Results = np.vstack([t, f]).T
Results[0, 0]

In [ ]:
evals_result = {}
model = xgb.train(params, dtrain,                   
                  num_boost_round=FIXED_PARAMS['num_boost_round'],
                  evals =[(dtrain, 'train'),(dtest, 'test')],
                  early_stopping_rounds=FIXED_PARAMS['early_stopping_rounds'],
                  evals_result=evals_result)

In [ ]:
score = model.best_score
score

In [ ]:
max(evals_result['test']['auc'])
#xgb.plot_metric(evals_result, metric='auc_mu') 


In [ ]:
stop = len(evals_result['test']['auc'])
epochs = np.linspace(1, stop, num=67)
plt.plot(epochs, evals_result['train']['auc'], label='AUC')
plt.plot(epochs, evals_result['test']['auc'], label='AUC')
plt.legend(loc="best")
plt.show()

In [ ]:
y_hat= np.array([[0.6319051,  0.4340122,  0.42533597],
             [0.6319051,  0.4340122,  0.42533597],
             [0.6319051,  0.4340122,  0.42533597],
             [0.6319051,  0.4340122,  0.42533597],
             [0.6319051, 0.4340122, 0.42533597]])
y_hat.argmax(axis=1)

In [ ]:
from sklearn.metrics import f1_score, recall_score

def xgb_f1_score(y_hat, data):
    y_true = data.get_label() 
    #y_hat = y_hat.reshape(y_hat.shape[0], 3) 
    #assert y_hat.shape == (238, 3)
    print(y_hat[300:305])
    y_hat = y_hat.argmax(axis=1)#.astype(np.float32)  
    print(y_hat.shape)
    print((y_true[300:305]), 'test', y_hat[300:305])
    print (len(y_true), 'test', len(y_hat))
    return 'f1_macro', f1_score(y_true, y_hat, average='macro')
    
def xgb_recall_score(y_hat, data): 
    y_true = data.get_label()
    y_hat = y_hat.reshape(-1, 3)
    y_hat = y_hat.argmax(axis=0).astype(np.float32)
    return 'recall', recall_score(y_true, y_hat, average='macro')
  
evals_result = {}
model = xgb.train(params, dtrain,                   
                  num_boost_round=FIXED_PARAMS['num_boost_round'],
                  evals =[(dtrain, 'train'),(dtest, 'test')],
                  #evals =[(dtest, 'test')],
                  feval=xgb_f1_score,
                  early_stopping_rounds=FIXED_PARAMS['early_stopping_rounds'],
                  evals_result=evals_result)
 

In [ ]:
print(f"train score:'{max(evals_result['train']['f1_macro'])}'") 
print(f"test score:'{max(evals_result['test']['f1_macro'])}'") 
#print(f"train auc score:'{max(evals_result['train']['auc'])}'") 
#print(f"test auc score:'{max(evals_result['test']['auc'])}'") 
#print(f"'{id_to_interpretation[actual]}' predicted as '{id_to_interpretation[predicted]}' : {conf_mat[actual, predicted]} examples.")
        
evals_result

In [ ]:
def plot_xbg(evals_result = evals_result):
    stop = len(evals_result['test']['f1_macro'])
    epochs = np.linspace(1, stop, num=stop)
    plt.plot(epochs, evals_result['train']['f1_macro'], label='train f1_macro')
    plt.plot(epochs, evals_result['test']['f1_macro'], label='test f1_macro')
    plt.legend(loc="best")
    plt.show()
plot_xbg(evals_result)

In [ ]:
y_train_predict = model.predict(dtrain)
print('Train accuracy', f1_score(y_train, y_train_predict, average = 'macro'))

In [ ]:
%run Utility.ipynb

In [ ]:
plot_conf_matrix(y_train, y_train_predict)

In [ ]:
xgb_pred = model.predict(dtest)
#xgb_pred = xgb_pred.argmax(axis = 1)
xgb_pred.shape
xgb_pred
xgb_F1 = recall_score(y_valid, xgb_pred, average = 'macro')
xgb_F1

In [ ]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_valid, xgb_pred)
fig, ax = plt.subplots(figsize=(5,5))
target_names = ['Negative', 'Positive', 'Intermediate']
sns.heatmap(conf_mat, annot=True, fmt='d', cmap=plt.cm.Blues, xticklabels=target_names, yticklabels=target_names) #cmap=plt.cm.Blues,
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
np.array(y_valid)

In [ ]:
lgb_pred

## Try out skopt.BayesSearchCV

In [ ]:
import skopt
from sklearn.metrics import f1_score
from skopt.space import Real, Categorical, Integer
import lightgbm as lgb
#patch below to solve TypeError: __init__() got an unexpected keyword argument 'iid'
def bayes_search_CV_init(self, estimator, search_spaces, optimizer_kwargs=None,
                         n_iter=50, scoring=None, fit_params=None, n_jobs=1,
                         n_points=1, iid=True, refit=True, cv=None, verbose=0,
                         pre_dispatch='2*n_jobs', random_state=None,
                         error_score='raise', return_train_score=False):

        self.search_spaces = search_spaces
        self.n_iter = n_iter
        self.n_points = n_points
        self.random_state = random_state
        self.optimizer_kwargs = optimizer_kwargs
        self._check_search_space(self.search_spaces)
        self.fit_params = fit_params

        super(skopt.BayesSearchCV, self).__init__(
             estimator=estimator, scoring=scoring,
             n_jobs=n_jobs, refit=refit, cv=cv, verbose=verbose,
             pre_dispatch=pre_dispatch, error_score=error_score,
             return_train_score=return_train_score)
        
skopt.BayesSearchCV.__init__ = bayes_search_CV_init

In [ ]:
'''opt = BayesSearchCV(
    SVC(),
    {
        'C': (1e-6, 1e+6, 'log-uniform'),
        'gamma': (1e-6, 1e+1, 'log-uniform'),
        'degree': (1, 8),  # integer valued parameter
        'kernel': ['linear', 'poly', 'rbf'],  # categorical parameter
    },
    n_iter=32,
    cv=3
)'''

In [ ]:
SEARCH_PARAMS = {'learning_rate': 0.3,
                 #'num_iterations': 200,
                 'max_depth': 6,                 
                 #'early_stopping_round': 20,  
                 #'min_child_weight': 1,
                 'colsample_bytree': 0.8,
                 'subsample': 0.8,                  
                 'gamma': 0                 
                } 
SEARCH_PARAMS

In [ ]:

SPACE = [
    skopt.space.Real(0.01, 0.2, name='learning_rate', prior='log-uniform'),
    #skopt.space.Integer(50, 500, name='num_iterations'),
    skopt.space.Integer(1, 30, name='max_depth'),    
    #skopt.space.Integer(20, 50, name='early_stopping_round'),
    #skopt.space.Real(1, 5, name='min_child_weight', prior='uniform'), 
    skopt.space.Real(0.1, 1.0, name='colsample_bytree', prior='uniform'),    
    skopt.space.Real(0.1, 1.0, name='subsample', prior='uniform'), 
    skopt.space.Real(0, 2.0, name='gamma', prior='uniform')    
    ]

In [ ]:
def xgb_f1_score(y_hat, data):
    y_true = data.get_label()
    #y_hat = y_hat.reshape(3, -1).T    
    y_hat = y_hat.argmax(axis=1)
    return 'f1_macro', f1_score(y_true, y_hat, average='macro') 

dtrain = xgb.DMatrix(X_train, label=y_train, weight = y_train_weigth)
dtest = xgb.DMatrix(X_valid, label=y_valid, weight = y_test_weigth)
def cv_evaluate(search_params):     
    params = {'num_class':3,
              'objective': 'multi:softprob',  
              'reg_lambda': 1,
              'disable_default_eval_metric': 1,
              **search_params}  
    num_boost_round = 1000
    cv_dict = xgb.cv(params, dtrain, nfold= 5, num_boost_round=num_boost_round,
                     early_stopping_rounds=30, 
                     seed=42, feval=xgb_f1_score)
    return max(cv_dict['test-f1_macro-mean'])
 
score = cv_evaluate(SEARCH_PARAMS)
score


In [ ]:
@skopt.utils.use_named_args(SPACE)
def objective(**params):
    return -1.0 * cv_evaluate(params)

In [ ]:
results = skopt.forest_minimize(objective, SPACE, n_calls=30, n_random_starts=10, random_state=0)
best_cv_f1 = -1.0 * results.fun
best_params_cv = results.x

print('best result: ', best_cv_f1)
print('best parameters: ', best_params_cv)

In [ ]:
#final_params_cv['num_iterations'] = best_params_cv[2]
#final_params_cv['early_stopping_round'] = best_params_cv[3]
#final_params_cv['min_data_in_leaf'] = best_params_cv[5]

final_params_cv = {'num_class':3,
                   'objective': 'multi:softprob',  
                   'reg_lambda': 1,
                   'disable_default_eval_metric': 1,         
          } 
final_params_cv['learning_rate'] = best_params_cv[0]
final_params_cv['max_depth'] = best_params_cv[1]
#final_params_cv['num_iterations'] = best_params_cv[2]
#final_params_cv['early_stopping_round'] = best_params_cv[3]
 
final_params_cv['colsample_bytree'] = best_params_cv[2]
final_params_cv['subsample'] = best_params_cv[3]
final_params_cv['gamma'] = best_params_cv[4] 

In [ ]:
final_params_cv

In [ ]:
evals_result = {}

clf_cv = xgb.train(final_params_cv, dtrain,
                   num_boost_round=num_boost_round,
                   early_stopping_rounds=30, 
                   evals =[(dtrain, 'train'),(dtest, 'test')],
                   feval=xgb_f1_score,
                   evals_result=evals_result)

plot_xbg(evals_result = evals_result)

In [ ]:
print(np.unique(np.array(y_valid)))
print(np.unique(np.array(y_train)))
print(np.unique(np.array(labels)))
print(len(np.array(y_valid)))
print(len(np.array(y_train)))

In [ ]:
xgb_pred_cv = clf_cv.predict(dtest)
xgb_pred_cv

In [ ]:
#xgb_pred_cv = clf_cv.predict(dtrain)
xgb_pred_cv = xgb_pred_cv.argmax(axis = 1)
xgb_pred_cv.shape


In [ ]:
xgb_F1_cv = f1_score(y_valid, xgb_pred_cv, average = 'macro')
xgb_F1_cv

In [ ]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(np.array(y_valid), xgb_pred_cv)
fig, ax = plt.subplots(figsize=(5,5))
target_names = ['Negative', 'Positive', 'Intermediate']
sns.heatmap(conf_mat, annot=True, fmt='d', cmap=plt.cm.Blues, xticklabels=target_names, yticklabels=target_names) #cmap=plt.cm.Blues,
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()


In [ ]:
# try out gridsearch + lgbclassfier with parameters trained with lgb.cv
grid_param = {}

for key, value in final_params_cv.items():
    temp = []
    temp.append(value)
    grid_param[key]=temp
grid_param
grid_param.pop('early_stopping_round')

In [ ]:
grid_param = {'objective': ['multiclass'],
              'learning_rate': [0.4],
              'max_depth': [9],
              'n_estimators': [75],
              'min_child_samples': [5],
              'min_child_weight': [0.0001],
              'scale_pos_weight': [0.1]}

In [ ]:
# use gridsearch + lgbclassfier
model = lgb.LGBMClassifier()
gs = GridSearchCV(model, grid_param, cv=5, scoring=['f1_micro','precision_micro','recall_micro'], refit='f1_micro')
gs = gs.fit(X_train, y_train)
best_model = gs.best_estimator_
pred = best_model.predict(X_valid)
print('Gridsearch best F1_micro score (based on logistic regression probabilities) =', gs.best_score_)
print('Model best F1 score =', f1_score(y_valid, pred, average='macro'))

In [ ]:
'''def plot_conf_matrix(y_true, y_hat):
    conf_mat = confusion_matrix(np.array(y_true), y_hat)
    fig, ax = plt.subplots(figsize=(5,5))
    target_names = ['Negative', 'Positive', 'Intermediate']
    sns.heatmap(conf_mat, annot=True, fmt='d', cmap=plt.cm.Blues, xticklabels=target_names, yticklabels=target_names) #cmap=plt.cm.Blues,
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()'''


In [ ]:
#plot_conf_matrix(y_valid, pred)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(np.array(y_valid), pred)
target_names = ['Negative', 'Positive', 'Intermediate']
cmp = ConfusionMatrixDisplay(cm, display_labels=target_names)
cmp.plot(cmap=plt.cm.Blues)

In [ ]:
#from Utility import plot_conf_matrix
%run Utility.ipynb
test = plot_conf_matrix(y_valid, pred)

## Tuning parameters with skopt using lgb.train

In [ ]:
import skopt

In [ ]:
SEARCH_PARAMS = {'learning_rate': 0.4,
                 'max_depth': 15,
                 'max_bin':300,
                 'num_leaves': 300,
                 'min_sum_hessian_in_leaf': 0.001,
                 'scale_pos_weight': 0.1,
                 'feature_fraction': 0.8,
                 'subsample': 0.2
                } 

In [ ]:
'''SEARCH_PARAMS = {'learning_rate': 0.4,
                 'max_depth': 15,
                 'num_iterations': 20,
                 'min_data_in_leaf':5,
                 'min_sum_hessian_in_leaf': 0.001,
                 'scale_pos_weight': 0.1
                }'''

In [ ]:
from sklearn.metrics import f1_score

def lgb_f1_score(y_hat, data):
        y_true = data.get_label()
        y_hat = y_hat.reshape(3, -1).T
        y_hat = y_hat.argmax(axis=1)
        return 'f1', f1_score(y_true, y_hat, average='macro'), True
    
def train_evaluate(search_params):     
    #X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=1234)

    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)

    params = {'metric':'None',
              'num_class':3,
              'objective': 'multiclass', 
              'force_col_wise': True, 
              **search_params}  
   
    evals_result = {}
    
    model = lgb.train(params, train_data, 
                      valid_sets=[valid_data, train_data], 
                      valid_names=['valid', 'train'], 
                      num_boost_round=300,
                      early_stopping_rounds=30,
                      feval=lgb_f1_score, evals_result=evals_result)
    score = model.best_score['valid']['f1']
    return score

In [ ]:
score = train_evaluate(SEARCH_PARAMS)


In [ ]:
'''SPACE = [
    skopt.space.Real(0.01, 0.5, name='learning_rate', prior='log-uniform'),
    skopt.space.Integer(1, 45, name='max_depth'),
    skopt.space.Integer(25, 1500, name='num_iterations'),
    skopt.space.Integer(5, 30, name='min_data_in_leaf'),
    skopt.space.Real(0.0001, 0.005, name='min_sum_hessian_in_leaf', prior='uniform'),
    skopt.space.Real(0.1, 10, name='scale_pos_weight', prior='uniform')]
 '''

In [ ]:
SPACE = [
    skopt.space.Real(0.01, 0.5, name='learning_rate', prior='log-uniform'),
    skopt.space.Integer(1, 30, name='max_depth'),
    skopt.space.Integer(300, 1000, name='max_bin'),
    skopt.space.Integer(500, 1000, name='num_leaves'),
    skopt.space.Real(0.0001, 0.005, name='min_sum_hessian_in_leaf', prior='uniform'), 
    skopt.space.Real(0.1, 10, name='scale_pos_weight', prior='uniform'),
    skopt.space.Real(0.1, 1.0, name='feature_fraction', prior='uniform'),    
    skopt.space.Real(0.1, 1.0, name='subsample', prior='uniform')]

In [ ]:
@skopt.utils.use_named_args(SPACE)
def objective(**params):
    return -1.0 * train_evaluate(params)

In [ ]:
results = skopt.forest_minimize(objective, SPACE, n_calls=30, n_random_starts=10)#, random_state= 0)
best_auc = -1.0 * results.fun
best_params = results.x

print('best result: ', best_auc)
print('best parameters: ', best_params)

In [ ]:
final_params = {'metric':'None',
                'num_class':3,
                'objective': 'multiclass',
                'force_col_wise': True, 
          }  

In [ ]:
final_params['learning_rate'] = best_params[0]
final_params['max_depth'] = best_params[1]
#final_params['max_bin'] = best_params[2]
final_params['num_leaves'] = best_params[3]
final_params['min_sum_hessian_in_leaf'] = best_params[4]
final_params['scale_pos_weight'] = best_params[5]
final_params['feature_fraction'] = best_params[6]
final_params['subsample'] = best_params[7] 

In [ ]:
'''final_params['learning_rate'] = best_params[0]
final_params['max_depth'] = best_params[1]
final_params['num_iterations'] = best_params[2]
final_params['min_data_in_leaf'] = best_params[3]
final_params['min_sum_hessian_in_leaf'] = best_params[4]
final_params['scale_pos_weight'] = best_params[5]'''

In [ ]:

final_params

In [ ]:
evals_result = {}

clf = lgb.train(final_params, train_data, valid_sets=[valid_data, train_data], valid_names=['valid', 'train'], feval=lgb_f1_score, evals_result=evals_result)

lgb.plot_metric(evals_result, metric='f1')

In [ ]:
lgb_pred = clf.predict(X_valid)
lgb_pred = lgb_pred.argmax(axis = 1)
lgb_pred.shape
lgb_pred
lgb_F1 = f1_score(y_valid, lgb_pred, average = 'macro')
lgb_F1

In [ ]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_valid, lgb_pred)
fig, ax = plt.subplots(figsize=(5,5))
target_names = ['Negative', 'Positive', 'Intermediate']
sns.heatmap(conf_mat, annot=True, fmt='d', cmap=plt.cm.Blues, xticklabels=target_names, yticklabels=target_names) #cmap=plt.cm.Blues,
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()